In [1]:
# Use DeepSurv from the repo
import lasagne
import deepsurv
from deepsurv import deep_surv, utils

from deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger
from deepsurv import viz

import numpy as np
import pandas as pd

from sklearn.utils import resample 
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
from lifelines import CoxPHFitter
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from bayes_opt import bayesian_optimization
import optunity
import h5py

In [11]:
# Cox Fitting
cf = CoxPHFitter()
cf.fit(train_df, 'Time', event_col='Event')

cf.print_summary(decimals = 3)  # access the results using cf.summary

<lifelines.CoxPHFitter: fitted with 1048 observations, 617 censored>
      duration col = 'Time'
         event col = 'Event'
number of subjects = 1048
  number of events = 431
partial log-likelihood = -2577.852
  time fit was run = 2019-10-07 06:35:53 UTC

---
    coef exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
1  0.036     1.037     0.005           0.027           0.046                1.027                1.047
2  0.098     1.103     0.111          -0.120           0.315                0.887                1.371
3 -0.062     0.940     0.011          -0.083          -0.042                0.920                0.959
4  0.919     2.507     0.108           0.707           1.132                2.027                3.101
5  0.249     1.283     0.104           0.046           0.452                1.047                1.571

       z       p  -log2(p)
1  7.633 <0.0005    45.313
2  0.882   0.378     1.405
3 -5.898 <0.0005    28.017
4  8.475 <0

In [12]:
#get CI to bootstrap
# configure bootstrap
n_iterations = 1000
n_size = int(len(train_df) * 0.50)

# run bootstrap
stats = list()
for i in range(n_iterations):
    # prepare train and test sets
    train = resample(train_df, n_samples=n_size)
    # fit model
    cf = CoxPHFitter()
    cf.fit(train, 'Time', event_col='Event')
    # evaluate model
    score = cf.score_
    stats.append(score)

# confidence intervals
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))

95.0 confidence interval 72.3% and 78.7%


In [8]:
def evaluate_model(model, dataset, bootstrap = False):
    def ci(model):
        def cph_ci(x, t, e, **kwargs):
            return concordance_index(t,-model.predict_partial_hazard(x),e,)
        return cph_ci


    metrics = {}

    # Calculate c_index
    metrics['c_index'] = ci(model)(**dataset)
    if bootstrap:
        metrics['c_index_bootstrap'] = utils.bootstrap_metric(ci(model), dataset)
        
    return metrics

# WHAS

In [9]:
import logging

import uuid
import time
localtime   = time.localtime()
TIMESTRING  = time.strftime("%m%d%Y%M", localtime)

DURATION_COL = 'time'
EVENT_COL = 'censor'
    

if __name__ == '__main__':

    datasets = utils.load_datasets('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/whas_train_test.h5')

    # Train CPH model
    print("Training CPH Model")
    train_df = utils.format_dataset_to_df(datasets['train'], DURATION_COL, EVENT_COL)
    cf = CoxPHFitter()
    results = cf.fit(train_df, duration_col=DURATION_COL, event_col=EVENT_COL)
    cf.print_summary()
    print("Train Likelihood: " + str(cf._log_likelihood))

    if 'valid' in datasets:
        metrics = evaluate_model(cf, datasets['valid'])
        print("Valid metrics: " + str(metrics))

    if 'test' in datasets:
        metrics = evaluate_model(cf, datasets['test'], bootstrap=True)
        print("Test metrics: " + str(metrics))



Training CPH Model
<lifelines.CoxPHFitter: fitted with 1310 observations, 758 censored>
      duration col = 'time'
         event col = 'censor'
number of subjects = 1310
  number of events = 552
partial log-likelihood = -3281.46
  time fit was run = 2019-11-06 08:08:20 UTC

---
   coef exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
0  1.73      5.65      0.09            1.55            1.92                 4.69                 6.79
1  0.03      1.03      0.00            0.03            0.04                 1.03                 1.04
2  0.05      1.06      0.10           -0.14            0.24                 0.87                 1.27
3 -0.05      0.95      0.01           -0.07           -0.03                 0.94                 0.97
4  0.76      2.13      0.10            0.57            0.95                 1.76                 2.58
5  0.27      1.32      0.09            0.09            0.45                 1.10                 1.57

    

C:\Users\ASUS\Anaconda3\lib\site-packages\lifelines\utils\__init__.py:920: ConvergenceWarning: Column 0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['censor'].astype(bool)
>>> print(df.loc[events, '0'].var())
>>> print(df.loc[~events, '0'].var())

A very low variance means that the column 0 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression 
  warnings.warn(warning_text, ConvergenceWarning)


Test metrics: {'c_index': 0.8160032357005629, 'c_index_bootstrap': {'mean': 0.8180394000479096, 'confidence_interval': (0.8145077431219854, 0.8215710569738339)}}


# SUPPORT

In [10]:
if __name__ == '__main__':

    datasets = utils.load_datasets('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/support_train_test.h5')

    # Train CPH model
    print("Training CPH Model")
    train_df = utils.format_dataset_to_df(datasets['train'], 'Time', 'Event')
    cf = CoxPHFitter()
    results = cf.fit(train_df, duration_col='Time', event_col='Event')
    cf.print_summary(decimals = 4)
    print("Train Likelihood: " + str(cf._log_likelihood))

    if 'valid' in datasets:
        metrics = evaluate_model(cf, datasets['valid'])
        print("Valid metrics: " + str(metrics))

    if 'test' in datasets:
        metrics = evaluate_model(cf, datasets['test'], bootstrap=True)
        print("Test metrics: " + str(metrics))


Training CPH Model
<lifelines.CoxPHFitter: fitted with 7098 observations, 2275 censored>
      duration col = 'Time'
         event col = 'Event'
number of subjects = 7098
  number of events = 4823
partial log-likelihood = -39956.9854
  time fit was run = 2019-12-24 02:37:32 UTC

---
      coef exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
0   0.0142    1.0143    0.0010          0.0122          0.0162               1.0123               1.0163
1  -0.0935    0.9107    0.0293         -0.1510         -0.0361               0.8598               0.9646
2   0.0241    1.0244    0.0119          0.0007          0.0475               1.0007               1.0486
3   0.0197    1.0199    0.0238         -0.0270          0.0664               0.9734               1.0687
4  -0.0469    0.9541    0.0398         -0.1250          0.0311               0.8825               1.0316
5   0.1321    1.1412    0.0784         -0.0217          0.2858               0.9786  

# METABRIC

In [9]:
if __name__ == '__main__':

    datasets = utils.load_datasets('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/metabric_IHC4_clinical_train_test.h5')

    # Train CPH model
    print("Training CPH Model")
    train_df = utils.format_dataset_to_df(datasets['train'], 'Time', 'Event')
    cf = CoxPHFitter()
    results = cf.fit(train_df, duration_col='Time', event_col='Event')
    cf.print_summary(decimals = 4)
    print("Train Likelihood: " + str(cf._log_likelihood))

    if 'valid' in datasets:
        metrics = evaluate_model(cf, datasets['valid'])
        print("Valid metrics: " + str(metrics))

    if 'test' in datasets:
        metrics = evaluate_model(cf, datasets['test'], bootstrap=True)
        print("Test metrics: " + str(metrics))


Training CPH Model
<lifelines.CoxPHFitter: fitted with 1523 observations, 636 censored>
      duration col = 'Time'
         event col = 'Event'
number of subjects = 1523
  number of events = 887
partial log-likelihood = -5752.2917
  time fit was run = 2019-12-24 02:31:53 UTC

---
     coef exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
0  0.0422    1.0431    0.0471         -0.0501          0.1345               0.9511               1.1440
1 -0.0698    0.9325    0.0381         -0.1446          0.0049               0.8654               1.0049
2  0.1023    1.1077    0.0260          0.0514          0.1532               1.0527               1.1656
3  0.3145    1.3696    0.1041          0.1104          0.5186               1.1167               1.6797
4  0.1849    1.2031    0.0774          0.0333          0.3366               1.0338               1.4001
5 -0.2118    0.8091    0.0709         -0.3508         -0.0729               0.7041            

# GBSG

In [14]:
if __name__ == '__main__':

    datasets = utils.load_datasets('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/gbsg_cancer_train_test.h5')

    # Train CPH model
    print("Training CPH Model")
    train_df = utils.format_dataset_to_df(datasets['train'], DURATION_COL, EVENT_COL)
    cf = CoxPHFitter()
    results = cf.fit(train_df, duration_col=DURATION_COL, event_col=EVENT_COL)
    cf.print_summary(decimals = 5)
    print("Train Likelihood: " + str(cf._log_likelihood))

    if 'valid' in datasets:
        metrics = evaluate_model(cf, datasets['valid'])
        print("Valid metrics: " + str(metrics))

    if 'test' in datasets:
        metrics = evaluate_model(cf, datasets['test'], bootstrap=True)
        print("Test metrics: " + str(metrics))


Training CPH Model
<lifelines.CoxPHFitter: fitted with 1546 observations, 578 censored>
      duration col = 'time'
         event col = 'censor'
number of subjects = 1546
  number of events = 968
partial log-likelihood = -6570.24803
  time fit was run = 2019-11-06 08:12:10 UTC

---
      coef exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
0 -0.31716   0.72821   0.08392        -0.48163        -0.15269              0.61777              0.85840
1  0.33792   1.40203   0.04969         0.24054         0.43530              1.27193              1.54543
2  0.25593   1.29166   0.11225         0.03592         0.47594              1.03658              1.60952
3  0.00256   1.00257   0.00419        -0.00566         0.01078              0.99436              1.01084
4  0.05572   1.05730   0.00547         0.04500         0.06644              1.04603              1.06870
5 -0.00026   0.99974   0.00014        -0.00053         0.00000              0.99947   

In [11]:
#nwtco 데이터 불러오기
nwtco = pd.read_csv('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/nwtco.csv')
train_df = nwtco.drop('seqno', axis = 1)
train_df = train_df.rename(columns = {'rel':'Event', 'edrel':'Time'})

#censored number
c_number = train_df['Event'].value_counts()[0]
feat_num = len(list(train_df.columns))
total_number = len(train_df['Event'])
c_rate = (c_number / total_number) * 100

print("total_number : ", total_number)
print("feature_number : ", feat_num)
print("censored_number : ", c_number)
print("censored_rate : " , c_rate)

#data split
train_df, test_df = train_test_split(train_df, test_size = 0.2)
train_df, val_df = train_test_split(train_df, test_size = 0.2)

total_number :  4028
feature_number :  8
censored_number :  3457
censored_rate :  85.82423038728898


In [12]:
# Cox Fitting
cf = CoxPHFitter()
cf.fit(train_df, 'Time', event_col='Event')

cf.print_summary(decimals = 4)  # access the results using cf.summary

<lifelines.CoxPHFitter: fitted with 2577 observations, 2216 censored>
      duration col = 'Time'
         event col = 'Event'
number of subjects = 2577
  number of events = 361
partial log-likelihood = -2667.1964
  time fit was run = 2019-12-24 02:39:05 UTC

---
                coef exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
instit        0.1526    1.1649    0.1824         -0.2049          0.5102               0.8147               1.6656
histol        1.4946    4.4576    0.1703          1.1608          1.8284               3.1925               6.2239
stage         0.3401    1.4050    0.0511          0.2399          0.4403               1.2711               1.5531
study        -0.0424    0.9585    0.1065         -0.2512          0.1664               0.7779               1.1810
age           0.0054    1.0054    0.0016          0.0023          0.0085               1.0023               1.0085
in.subcohort -0.1521    0.8589    0.1489      

In [53]:
#get CI to bootstrap
# configure bootstrap
n_iterations = 1000
n_size = int(len(train_df) * 0.50)

# run bootstrap
stats = list()
for i in range(n_iterations):
    # prepare train and test sets
    train = resample(train_df, n_samples=n_size)
    # fit model
    cf = CoxPHFitter()
    cf.fit(train, 'Time', event_col='Event')
    # evaluate model
    score = cf.score_
    stats.append(score)

# confidence intervals
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))

95.0 confidence interval 67.7% and 75.4%


# non-linear

In [2]:
#generate data
def generate_data(treatment_group = False):
    np.random.seed(123)
    sd = deepsurv.datasets.SimulatedData(5, num_features = 9,
        treatment_group = treatment_group)
    train_data = sd.generate_data(1000,method = 'gaussian')
    valid_data = sd.generate_data(400,method = 'gaussian')
    test_data = sd.generate_data(400,method = 'gaussian')

    return train_data, valid_data, test_data

train_data, valid_data, test_data = generate_data()

In [5]:
if __name__ == '__main__':

    # Train CPH model
    print("Training CPH Model")
    train_df = utils.format_dataset_to_df(train_data, 't', 'e')
    cf = CoxPHFitter()
    results = cf.fit(train_df, duration_col='t', event_col='e')
    cf.print_summary()
    print("Train Likelihood: " + str(cf._log_likelihood))

    metrics = evaluate_model(cf, valid_data)
    print("Valid metrics: " + str(metrics))

    metrics = evaluate_model(cf, test_data, bootstrap=True)
    print("Test metrics: " + str(metrics))

Training CPH Model
<lifelines.CoxPHFitter: fitted with 1000 observations, 80 censored>
      duration col = 't'
         event col = 'e'
number of subjects = 1000
  number of events = 920
partial log-likelihood = -5629.63
  time fit was run = 2019-11-06 05:04:26 UTC

---
   coef exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
0 -0.18      0.83      0.05           -0.28           -0.08                 0.75                 0.92
1  0.04      1.04      0.05           -0.07            0.14                 0.93                 1.15
2  0.11      1.12      0.06           -0.00            0.23                 1.00                 1.26
3 -0.02      0.98      0.06           -0.14            0.09                 0.87                 1.09
4  0.00      1.00      0.06           -0.11            0.11                 0.90                 1.12
5 -0.01      0.99      0.06           -0.12            0.11                 0.89                 1.11
6  0.07      1

# linear

In [6]:
def generate_data(treatment_group = False):
    np.random.seed(123)
    sd = deepsurv.datasets.SimulatedData(5, num_features = 9,
        treatment_group = treatment_group)
    train_data = sd.generate_data(1000,method = 'linear')
    valid_data = sd.generate_data(400,method = 'linear')
    test_data = sd.generate_data(400,method = 'linear')

    return train_data, valid_data, test_data

train_data, valid_data, test_data = generate_data()

In [7]:
if __name__ == '__main__':

    # Train CPH model
    print("Training CPH Model")
    train_df = utils.format_dataset_to_df(train_data, 't', 'e')
    cf = CoxPHFitter()
    results = cf.fit(train_df, duration_col='t', event_col='e')
    cf.print_summary()
    print("Train Likelihood: " + str(cf._log_likelihood))

    metrics = evaluate_model(cf, valid_data)
    print("Valid metrics: " + str(metrics))

    metrics = evaluate_model(cf, test_data, bootstrap=True)
    print("Test metrics: " + str(metrics))

Training CPH Model
<lifelines.CoxPHFitter: fitted with 1000 observations, 172 censored>
      duration col = 't'
         event col = 'e'
number of subjects = 1000
  number of events = 828
partial log-likelihood = -4811.95
  time fit was run = 2019-11-06 05:04:39 UTC

---
   coef exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
0  0.80      2.22      0.06            0.67            0.92                 1.96                 2.52
1  2.03      7.61      0.08            1.87            2.19                 6.49                 8.91
2  0.21      1.23      0.06            0.08            0.33                 1.08                 1.39
3 -0.04      0.96      0.06           -0.16            0.08                 0.85                 1.09
4  0.04      1.04      0.06           -0.08            0.16                 0.92                 1.17
5 -0.03      0.97      0.06           -0.15            0.09                 0.86                 1.09
6  0.01      